# Genetic Program Example - adapted from TinyGP by Moshesipper

Import libraries

In [1]:
from random import random, randint, seed
from statistics import mean
from copy import deepcopy
import numpy as np

## Define parameters

We must now define our parameters, and allow for bloat control.

In [7]:
POP_SIZE        = 60   # population size
MIN_DEPTH       = 5    # minimal initial random tree depth
MAX_DEPTH       = 10   # maximal initial random tree depth
GENERATIONS     = 250  # maximal number of generations to run evolution
TOURNAMENT_SIZE = 5    # size of tournament for tournament selection
XO_RATE         = 0.9  # crossover rate 
PROB_MUTATION   = 0.2  # per-node mutation probability 

## Defining non-terminals and terminals

Define functions for non terminal set - this can be done natively in functional programming languages.

In [3]:
def add(x, y): return np.add(x, y)
def sub(x, y): return np.subtract(x, y)
def mul(x, y): return np.multiply(x, y)
def cos(x, y): return np.cos(add(x,y))
#def div(x,y): return x/y # Consider what issues might arrise with this function

#Define terminal and non-terminal sets
FUNCTIONS = [add, sub, mul, cos]
TERMINALS = ['xx', 10,0,1,2, np.pi]

## Managing our dataset

In usual settings you will have a dataset which you are working from, much in the same way as in traditional ML tasks; however, for the assignment and for observing, we will consider a target function and a create a dataset from that.

In [4]:
def target_func(xx): # evolution's target
    return sum([x**2 + 2*x + 1 for x in xx])

In [5]:
def f1(xx):
    return sum([x**2 for x in xx])

def cosine(xx):
    return sum([10*np.cos(2*np.pi*x) for x in xx])


def f2(xx):
    return 10*len(xx) + sum([x**2 - 10*np.cos(2*np.pi*x) for x in xx])

In [6]:
def generate_dataset(): # generate 101 data points from target_func
    dataset = []
    for x in np.linspace(-5.12,5.12,100):
        for y in np.linspace(-5.12,5.12,100):
            #dataset.append([x,y, target_func([x,y])])
            #dataset.append([x,y, f1([x,y])])
            dataset.append([x,y, cosine([x,y])])
    return dataset

## Creating the genetic program class

In [8]:
class GPTree:
    def __init__(self, data = None, left = None, right = None):
        self.data  = data
        self.left  = left
        self.right = right
        
    def node_label(self): # string label
        if (self.data in FUNCTIONS):
            return self.data.__name__
        else: 
            return str(self.data)
    
    def print_tree(self, prefix = ""): # textual printout
        print("%s%s" % (prefix, self.node_label()))        
        if self.left:  self.left.print_tree (prefix + "   ")
        if self.right: self.right.print_tree(prefix + "   ")

    def compute_tree(self, x): 
        if (self.data in FUNCTIONS): 
            return self.data(self.left.compute_tree(x), self.right.compute_tree(x))
        elif self.data == 'xx': return x
        else: return self.data
            
    def random_tree(self, grow, max_depth, depth = 0): # create random tree using either grow or full method
        if depth < MIN_DEPTH or (depth < max_depth and not grow): 
            self.data = FUNCTIONS[randint(0, len(FUNCTIONS)-1)]
        elif depth >= max_depth:   
            self.data = TERMINALS[randint(0, len(TERMINALS)-1)]
        else: # intermediate depth, grow
            if random () > 0.5: 
                self.data = TERMINALS[randint(0, len(TERMINALS)-1)]
            else:
                self.data = FUNCTIONS[randint(0, len(FUNCTIONS)-1)]
        if self.data in FUNCTIONS:
            self.left = GPTree()          
            self.left.random_tree(grow, max_depth, depth = depth + 1)            
            self.right = GPTree()
            self.right.random_tree(grow, max_depth, depth = depth + 1)

    def mutation(self):
        if random() < PROB_MUTATION: # mutate at this node
            self.random_tree(grow = True, max_depth = 2)
        elif self.left: self.left.mutation()
        elif self.right: self.right.mutation() 

    def size(self): # tree size in nodes
        if self.data in TERMINALS: return 1
        l = self.left.size()  if self.left  else 0
        r = self.right.size() if self.right else 0
        return 1 + l + r

    def build_subtree(self): # count is list in order to pass "by reference"
        t = GPTree()
        t.data = self.data
        if self.left:  t.left  = self.left.build_subtree()
        if self.right: t.right = self.right.build_subtree()
        return t
                        
    def scan_tree(self, count, second): # note: count is list, so it's passed "by reference"
        count[0] -= 1            
        if count[0] <= 1: 
            if not second: # return subtree rooted here
                return self.build_subtree()
            else: # glue subtree here
                self.data  = second.data
                self.left  = second.left
                self.right = second.right
        else:  
            ret = None              
            if self.left  and count[0] > 1: ret = self.left.scan_tree(count, second)  
            if self.right and count[0] > 1: ret = self.right.scan_tree(count, second)  
            return ret

    def crossover(self, other): # xo 2 trees at random nodes
        if random() < XO_RATE:
            second = other.scan_tree([randint(1, other.size())], None) # 2nd random subtree
            self.scan_tree([randint(1, self.size())], second) # 2nd subtree "glued" inside 1st tree

## Fitness and selection

In [9]:
def fitness(individual, dataset): # inverse mean absolute error over dataset normalized to [0,1]
    return 1 / (1 + mean([abs(np.sum(individual.compute_tree(ds[0:-1])) - ds[-1]) for ds in dataset]))

In the example we are using we are using tournament based fitness. What benefits and negatives does tournament selection have?

In [10]:
def selection(population, fitnesses): # select one individual using tournament selection
    tournament = [randint(0, len(population)-1) for i in range(TOURNAMENT_SIZE)] # select tournament contenders
    tournament_fitnesses = [fitnesses[tournament[i]] for i in range(TOURNAMENT_SIZE)]
    return deepcopy(population[tournament[tournament_fitnesses.index(max(tournament_fitnesses))]]) 

Try to implement a roulette wheel selection for this and compare your results.

In [11]:
def roulette_selection(population, fitnesses):
    pass

In [16]:
def init_population(): # ramped half-and-half
    pop = []
    for md in range(MIN_DEPTH, MAX_DEPTH):
        for i in range(int(POP_SIZE/10)):
            t = GPTree()
            t.random_tree(grow = True, max_depth = md) # grow
            pop.append(t) 
        for i in range(int(POP_SIZE/10)):
            t = GPTree()
            t.random_tree(grow = False, max_depth = md) # full
            pop.append(t) 
    return pop

##  Main Loop

In [17]:
dataset = generate_dataset()
population = init_population() 
best_of_run = None
best_of_run_f = 0
best_of_run_gen = 0
print(len(population))
fitnesses = [fitness(population[i], dataset) for i in range(POP_SIZE)]

60


In [57]:
print(len(population))

60


In [ ]:
    # go evolution!
    for gen in range(GENERATIONS):        
        nextgen_population=[]
        
        for i in range(POP_SIZE):
            parent1 = selection(population, fitnesses)
            parent2 = selection(population, fitnesses)
            parent1.crossover(parent2)
            parent1.mutation()
            nextgen_population.append(parent1)
        population=nextgen_population
        fitnesses = [fitness(population[i], dataset) for i in range(POP_SIZE)]
        print("________________________")
        print("gen:", gen, ", best_of_run_f:", round(max(fitnesses),3), ", best_of_run_f:",best_of_run_f) 
        if max(fitnesses) > best_of_run_f:
            print('new_tree')
            best_of_run_f = max(fitnesses)
            best_of_run_gen = gen
            best_of_run = deepcopy(population[fitnesses.index(max(fitnesses))])
            best_of_run.print_tree()
        if best_of_run_f == 1: break
            
    
    print("\n\n_________________________________________________\nEND OF RUN\nbest_of_run attained at gen " + str(best_of_run_gen) +\
          " and has f=" + str(round(best_of_run_f,3)))
    best_of_run.print_tree()

________________________
gen: 0 , best_of_run_f: 0.111 , best_of_run_f: 0
new_tree
cos
   sub
      sub
         cos
            add
               add
                  mul
                     add
                        add
                           sub
                              10
                              1
                           mul
                              10
                              3.141592653589793
                        mul
                           sub
                              1
                              0
                           mul
                              xx
                              3.141592653589793
                     mul
                        cos
                           add
                              2
                              1
                           sub
                              0
                              10
                        mul
                           cos
                           

________________________
gen: 3 , best_of_run_f: 0.113 , best_of_run_f: 0.11263890539463957
________________________
gen: 4 , best_of_run_f: 0.113 , best_of_run_f: 0.11263890539463957
________________________
gen: 5 , best_of_run_f: 0.113 , best_of_run_f: 0.11263890539463957
________________________
gen: 6 , best_of_run_f: 0.113 , best_of_run_f: 0.11263890539463957
new_tree
cos
   cos
      mul
         sub
            cos
               mul
                  sub
                     sub
                        mul
                           cos
                              1
                              1
                           add
                              3.141592653589793
                              3.141592653589793
                        cos
                           add
                              3.141592653589793
                              3.141592653589793
                           add
                              10
                              2
      

________________________
gen: 9 , best_of_run_f: 0.113 , best_of_run_f: 0.11271458592190012
new_tree
cos
   cos
      mul
         sub
            sub
               cos
                  add
                     sub
                        add
                           add
                              mul
                                 mul
                                    mul
                                       sub
                                          10
                                          xx
                                       sub
                                          2
                                          1
                                    sub
                                       cos
                                          1
                                          10
                                       sub
                                          1
                                          10
                                 sub
        

________________________
gen: 10 , best_of_run_f: 0.113 , best_of_run_f: 0.11285403552405901
________________________
gen: 11 , best_of_run_f: 0.113 , best_of_run_f: 0.11285403552405901
________________________
gen: 12 , best_of_run_f: 0.113 , best_of_run_f: 0.11285403552405901
new_tree
cos
   cos
      cos
         add
            cos
               0
               sub
                  3.141592653589793
                  mul
                     xx
                     add
                        3.141592653589793
                        sub
                           xx
                           10
            mul
               sub
                  mul
                     1
                     3.141592653589793
                  add
                     1
                     0
               mul
                  add
                     10
                     3.141592653589793
                  sub
                     10
                     10
         sub
            cos

________________________
gen: 20 , best_of_run_f: 0.113 , best_of_run_f: 0.11338338317868626
________________________
gen: 21 , best_of_run_f: 0.113 , best_of_run_f: 0.11338338317868626
________________________
gen: 22 , best_of_run_f: 0.113 , best_of_run_f: 0.11338338317868626
________________________
gen: 23 , best_of_run_f: 0.113 , best_of_run_f: 0.11338338317868626
________________________
gen: 24 , best_of_run_f: 0.113 , best_of_run_f: 0.11338338317868626
________________________
gen: 25 , best_of_run_f: 0.113 , best_of_run_f: 0.11338338317868626
________________________
gen: 26 , best_of_run_f: 0.113 , best_of_run_f: 0.11338338317868626
________________________
gen: 27 , best_of_run_f: 0.113 , best_of_run_f: 0.11338338317868626
________________________
gen: 28 , best_of_run_f: 0.113 , best_of_run_f: 0.11338338317868626
________________________
gen: 29 , best_of_run_f: 0.113 , best_of_run_f: 0.11338338317868626
________________________
gen: 30 , best_of_run_f: 0.113 , best_of_run_

________________________
gen: 55 , best_of_run_f: 0.113 , best_of_run_f: 0.11377285709365158
________________________
gen: 56 , best_of_run_f: 0.113 , best_of_run_f: 0.11377285709365158
________________________
gen: 57 , best_of_run_f: 0.113 , best_of_run_f: 0.11377285709365158
________________________
gen: 58 , best_of_run_f: 0.114 , best_of_run_f: 0.11377285709365158
new_tree
cos
   sub
      add
         sub
            mul
               0
               10
            add
               xx
               xx
         sub
            mul
               2
               2
            add
               3.141592653589793
               10
      add
         add
            add
               0
               0
            sub
               0
               2
         add
            cos
               3.141592653589793
               1
            cos
               2
               0
   mul
      add
         sub
            sub
               xx
               0
            mul
   